---
title: JUNO Magnetic field data pipeline
---

In [ ]:
#| export
from datetime import timedelta
import polars as pl

from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

In [ ]:
#| hide
#| default_exp pipelines/juno/mag
%load_ext autoreload
%autoreload 2

## Downloading data 

In [ ]:
#| export
import pooch
from pooch import Unzip
from ids_finder.utils.basic import load_lbl, concat_partitions
from pipe import select, filter

In [ ]:
# | export
time_resolutions = ["1sec", "1min"]


def download_mag_data(
    start=None,
    end=None,
    ts: str = "1sec",  # time resolution
) -> list[str]:
    base_url = "https://pds-ppi.igpp.ucla.edu/ditdos/download?id=pds://PPI/JNO-SS-3-FGM-CAL-V1.0/DATA/CRUISE/SE"
    files = pooch.retrieve(
        url=f"{base_url}/{ts.upper()}",
        known_hash=None,
        path="../data/01_raw/",
        processor=Unzip(extract_dir=f"jno_ss_se_{ts}"),
    )
    return files


def load_mag_data(
    start: str | None = None,
    end: str | None = None,
    ts: str = "1sec",  # time resolution
) -> pl.DataFrame:
    files = download_mag_data(start, end, ts)

    data = pl.concat(
        files
        | filter(lambda x: x.endswith(".lbl"))
        | select(load_lbl)
        | select(pl.from_dataframe)
    )

    return data

## Preprocessing data

Convert all files from `lbl` format to `parquet` format for faster processing

In [ ]:
#| export
from ids_finder.utils.basic import concat_partitions

In [ ]:
# | export
def preprocess_mag_data(raw_data: pl.DataFrame) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Applying naming conventions for columns
    - Parsing and typing data
    - Changing storing format (from `lbl` to `parquet`)
    - Dropping useless columns
    """

    df_pl = (
        raw_data
        .lazy()
        .with_columns(time=pl.col("SAMPLE UTC").str.to_datetime("%Y %j %H %M %S %f"))
        .drop(["SAMPLE UTC", "DECIMAL DAY", "INSTRUMENT RANGE"])
        .sort("time")
        .collect()
    )
    return df_pl

### Processing data

In [ ]:
# | export
from ids_finder.utils.basic import partition_data_by_year

In [ ]:
#| export
def process_mag_data(
    raw_data: pl.DataFrame,
    ts: str = None,  # time resolution
    coord: str = None,
) -> pl.DataFrame | Dict[str, pl.DataFrame]:
    """
    Partitioning data, for the sake of memory
    """
    return partition_data_by_year(raw_data)

### Pipeline

In [ ]:
# | export
def create_pipeline(
    sat_id,
    ts: str = "1s",  # time resolution,
    tau: str = "60s",  # time window
    **kwargs,
) -> Pipeline:

    node_download_data = node(
        load_mag_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
        ),
        outputs=f"raw_mag_files_{ts}",
        name=f"download_{sat_id.upper()}_magnetic_field_data",
    )

    node_preprocess_data = node(
        preprocess_mag_data,
        inputs=dict(
            raw_data=f"raw_mag_{ts}",
        ),
        outputs=f"inter_mag_{ts}",
        name=f"preprocess_{sat_id.upper()}_magnetic_field_data",
    )
    
    node_process_data = node(
        process_mag_data,
        inputs=f"inter_mag_{ts}",
        outputs=f"primary_mag_rtn_{ts}",
        name=f"process_{sat_id.upper()}_magnetic_field_data",
    )
    
    node_extract_features = node(
        extract_features,
        inputs=[f"primary_mag_rtn_{ts}", "params:tau", "params:extract_params"],
        outputs=f"feature_tau_{tau}",
        name=f"extract_{sat_id}_features",
    )

    nodes = [
        node_download_data,
        node_preprocess_data,
        node_process_data,
        node_extract_features,
    ]

    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:tau": "params:tau",
            "params:extract_params": "params:jno_1s_params",
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
        },
    )
    return pipelines